In [128]:
import backtrader as bt
import yfinance as yf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import talib

In [129]:
data = yf.download("AAPL", start="2022-01-01", end="2024-01-01")
data

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-01-03,177.830002,182.880005,177.710007,182.009995,179.076569,104487900
2022-01-04,182.630005,182.940002,179.119995,179.699997,176.803833,99310400
2022-01-05,179.610001,180.169998,174.639999,174.919998,172.100861,94537600
2022-01-06,172.699997,175.300003,171.639999,172.000000,169.227936,96904000
2022-01-07,172.889999,174.139999,171.029999,172.169998,169.395187,86709100
...,...,...,...,...,...,...
2023-12-22,195.179993,195.410004,192.970001,193.600006,192.656189,37122800
2023-12-26,193.610001,193.889999,192.830002,193.050003,192.108856,28919300
2023-12-27,192.490005,193.500000,191.089996,193.149994,192.208359,48087700


In [130]:
data['RSI'] = talib.RSI(data['Close'], timeperiod=14)
data['RSI'].dropna()

Date
2022-01-24    15.097578
2022-01-25    14.138460
2022-01-26    14.091307
2022-01-27    13.831862
2022-01-28    41.330315
                ...    
2023-12-22    54.609714
2023-12-26    53.026883
2023-12-27    53.291938
2023-12-28    54.481470
2023-12-29    51.061838
Name: RSI, Length: 487, dtype: float64

In [131]:
# Buy when RSI < 30 (oversold), Sell when RSI > 70 (overbought)
data['Signal'] = np.where(data['RSI'] < 30, 1, 0)  
data['Signal'] = np.where(data['RSI'] > 70, -1, data['Signal'])

In [132]:
initial_cash = 10000
cash = initial_cash
position=0
portfolio_value=cash

trade_history=[]


In [133]:
for i in range(1,len(data)):
    
    signal=data['Signal'].iloc[i]
    price=data['Close'].iloc[i]

    if signal == 1 and position == 0:
        position = cash // price
        cash -= position * price
        trade_history.append(('BUY', data.index[i], price, position))

    elif signal == -1 and position > 0:
        cash += position * price
        trade_history.append(('SELL', data.index[i], price, position))
        position = 0

    portfolio_value = cash + position * price


data['Portfolio_Value'] = cash + (position * data['Close'])
        

In [134]:
data['Cumulative_Returns'] = (1 + data['Portfolio_Value'].pct_change()).cumprod()
print(data[['Portfolio_Value', 'Cumulative_Returns']].tail())


            Portfolio_Value  Cumulative_Returns
Date                                           
2023-12-22     11625.390747                 1.0
2023-12-26     11625.390747                 1.0
2023-12-27     11625.390747                 1.0
2023-12-28     11625.390747                 1.0
2023-12-29     11625.390747                 1.0


In [135]:
trade_history

[('BUY', Timestamp('2022-01-24 00:00:00'), 161.6199951171875, 61.0),
 ('SELL', Timestamp('2022-07-29 00:00:00'), 162.50999450683594, 61.0),
 ('BUY', Timestamp('2022-09-30 00:00:00'), 138.1999969482422, 72.0),
 ('SELL', Timestamp('2023-02-02 00:00:00'), 150.82000732421875, 72.0),
 ('BUY', Timestamp('2023-08-07 00:00:00'), 178.85000610351562, 61.0),
 ('SELL', Timestamp('2023-11-16 00:00:00'), 189.7100067138672, 61.0)]

In [136]:
data2= yf.download("AAPL", start="2023-06-01", end="2024-01-01",interval='1h')

[*********************100%%**********************]  1 of 1 completed


In [141]:
data2['RSI'] = talib.RSI(data2['Close'], timeperiod=14)
data2['RSI'].dropna()

Datetime
2023-06-05 09:30:00-04:00    88.110308
2023-06-05 10:30:00-04:00    88.950954
2023-06-05 11:30:00-04:00    89.446112
2023-06-05 12:30:00-04:00    83.929097
2023-06-05 13:30:00-04:00    72.404770
                               ...    
2023-12-29 11:30:00-05:00    35.346644
2023-12-29 12:30:00-05:00    35.346644
2023-12-29 13:30:00-05:00    38.127876
2023-12-29 14:30:00-05:00    44.654307
2023-12-29 15:30:00-05:00    43.250652
Name: RSI, Length: 1007, dtype: float64

In [143]:
# Buy when RSI < 30 (oversold), Sell when RSI > 70 (overbought)
data2['Signal'] = np.where(data2['RSI'] < 30, 1, 0)  
data2['Signal'] = np.where(data2['RSI'] > 70, -1, data2['Signal'])

In [145]:
initial_cash = 10000
cash = initial_cash
position=0
portfolio_value=cash

trade_history=[]


In [146]:
for i in range(1,len(data2)):
    
    signal=data2['Signal'].iloc[i]
    price=data2['Close'].iloc[i]

    if signal == 1 and position == 0:
        position = cash // price
        cash -= position * price
        trade_history.append(('BUY', data2.index[i], price, position))

    elif signal == -1 and position > 0:
        cash += position * price
        trade_history.append(('SELL', data2.index[i], price, position))
        position = 0

    portfolio_value = cash + position * price


data2['Portfolio_Value'] = cash + (position * data2['Close'])
        

In [147]:
data2['Cumulative_Returns'] = (1 + data2['Portfolio_Value'].pct_change()).cumprod()
print(data2[['Portfolio_Value', 'Cumulative_Returns']].tail())


                           Portfolio_Value  Cumulative_Returns
Datetime                                                      
2023-12-29 11:30:00-05:00      9147.301819            1.076317
2023-12-29 12:30:00-05:00      9147.301819            1.076317
2023-12-29 13:30:00-05:00      9157.641876            1.077534
2023-12-29 14:30:00-05:00      9183.961761            1.080631
2023-12-29 15:30:00-05:00      9176.442307            1.079746


In [148]:
trade_history

[('BUY',
  Timestamp('2023-08-02 11:30:00-0400', tz='America/New_York'),
  191.92999267578125,
  52.0),
 ('SELL',
  Timestamp('2023-08-23 09:30:00-0400', tz='America/New_York'),
  180.10499572753906,
  52.0),
 ('BUY',
  Timestamp('2023-09-06 13:30:00-0400', tz='America/New_York'),
  182.63429260253906,
  51.0),
 ('SELL',
  Timestamp('2023-10-06 11:30:00-0400', tz='America/New_York'),
  176.67010498046875,
  51.0),
 ('BUY',
  Timestamp('2023-10-20 10:30:00-0400', tz='America/New_York'),
  172.86990356445312,
  52.0),
 ('SELL',
  Timestamp('2023-11-01 14:30:00-0400', tz='America/New_York'),
  173.91000366210938,
  52.0),
 ('BUY',
  Timestamp('2023-12-27 09:30:00-0500', tz='America/New_York'),
  191.6385040283203,
  47.0)]

In [153]:
data3= yf.download("NVDA", start="2022-01-01", end="2024-01-01")

[*********************100%%**********************]  1 of 1 completed


In [158]:
data3['RSI'] = talib.RSI(data3['Close'], timeperiod=7)
data3['RSI'].dropna()

# Buy when RSI < 30 (oversold), Sell when RSI > 70 (overbought)
data3['Signal'] = np.where(data3['RSI'] < 20, 1, 0)  
data3['Signal'] = np.where(data3['RSI'] > 80, -1, data3['Signal'])

initial_cash = 10000
cash = initial_cash
position=0
portfolio_value=cash

trade_history=[]

for i in range(1,len(data3)):
    
    signal=data3['Signal'].iloc[i]
    price=data3['Close'].iloc[i]

    if signal == 1 and position == 0:
        position = cash // price
        cash -= position * price
        trade_history.append(('BUY', data3.index[i], price, position))

    elif signal == -1 and position > 0:
        cash += position * price
        trade_history.append(('SELL', data3.index[i], price, position))
        position = 0

    portfolio_value = cash + position * price


data3['Portfolio_Value'] = cash + (position * data3['Close'])
        
data3['Cumulative_Returns'] = (1 + data3['Portfolio_Value'].pct_change()).cumprod()
print(data3[['Portfolio_Value', 'Cumulative_Returns']].tail())


            Portfolio_Value  Cumulative_Returns
Date                                           
2023-12-22     10785.652782                 1.0
2023-12-26     10785.652782                 1.0
2023-12-27     10785.652782                 1.0
2023-12-28     10785.652782                 1.0
2023-12-29     10785.652782                 1.0


In [159]:
trade_history

[('BUY', Timestamp('2022-01-19 00:00:00'), 25.066999435424805, 398.0),
 ('SELL', Timestamp('2022-07-21 00:00:00'), 18.049999237060547, 398.0),
 ('BUY', Timestamp('2022-09-01 00:00:00'), 13.937000274658203, 517.0),
 ('SELL', Timestamp('2023-01-17 00:00:00'), 17.70199966430664, 517.0),
 ('BUY', Timestamp('2023-09-21 00:00:00'), 41.016998291015625, 223.0),
 ('SELL', Timestamp('2023-11-10 00:00:00'), 48.334999084472656, 223.0)]